## Действительно ли помогают неразмеченные данные?

Частичное обучение (semi-supervised learning) предлагает методы работы с выборками, в которых лишь для части объектов известны ответы. В статьях утверждается, что добавление неразмеченных данных позволяет повысить качество работы — давайте выясним, так ли это!

Наверное, проще всего добыть неразмеченные примеры, если речь идёт о работе с текстами или изображениями. Остановимся на текстах.

Будем работать с данными из соревнования Predict closed questions on Stack Overflow: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

Нас будет интересовать файл train-sample.csv — загрузите его. Будем решать бинарную задачу: отнесём объект к классу 1, если `OpenStatus == 'open'`, и к классу 0 иначе.

**Задание 1. (5 баллов)**

Загрузите данные и подготовьте выборку. В качестве признаков возьмите TF-IDF по BodyMarkdown с `min_df=10`; про целевую переменную написано выше. Выделите тестовую выборку из 5000 объектов.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack, csr_matrix
import warnings
import os
warnings.filterwarnings('ignore')

# Проверяем наличие файла
file_path = "train-sample.csv"
if not os.path.exists(file_path):
    print(f"Файл {file_path} не найден!")
    print("Текущая директория:", os.getcwd())
    print("Доступные CSV файлы:", [f for f in os.listdir() if f.endswith('.csv')])
    # Временно создадим тестовые данные для демонстрации
    print("\nСоздаю тестовые данные для демонстрации...")
    data_container = pd.DataFrame({
        'BodyMarkdown': ['sample text ' + str(i) for i in range(10000)],
        'OpenStatus': ['open' if i % 2 == 0 else 'closed' for i in range(10000)]
    })
else:
    data_container = pd.read_csv(file_path)

data_container['binary_label'] = (data_container['OpenStatus'] == 'open').astype(int)

textual_content = data_container['BodyMarkdown'].fillna('')
target_variable = data_container['binary_label']

text_vectorizer = TfidfVectorizer(min_df=10, max_features=5000)
vectorized_data = text_vectorizer.fit_transform(textual_content)

training_features, testing_features, training_labels, testing_labels = train_test_split(
    vectorized_data, target_variable, test_size=5000, random_state=42, stratify=target_variable
)

labeled_count = 500
labeled_features, unlabeled_features, labeled_labels, unlabeled_labels = train_test_split(
    training_features, training_labels, train_size=labeled_count, random_state=42, stratify=training_labels
)

def perform_model_evaluation(estimator, features_train, targets_train, features_test, targets_test, estimator_title):
    estimator.fit(features_train, targets_train)
    predicted_probs = estimator.predict_proba(features_test)[:, 1]
    roc_value = roc_auc_score(targets_test, predicted_probs)
    print(f"{estimator_title}: AUC-ROC = {roc_value:.4f}")
    return roc_value

print("\n1. Обучение исключительно на аннотированных экземплярах:")
baseline_classifier = LogisticRegression(random_state=42, max_iter=1000)
baseline_auc = perform_model_evaluation(
    baseline_classifier, labeled_features, labeled_labels, testing_features, testing_labels,
    "Логистическая регрессия (только размеченные)"
)

print("\n2. Self-training с неаннотированными данными (исключая тестовый набор):")

combined_features_case1 = vstack([labeled_features, unlabeled_features])
combined_labels_case1 = np.concatenate([labeled_labels, [-1] * unlabeled_features.shape[0]])

base_estimator = LogisticRegression(random_state=42, max_iter=1000)
semi_supervised_model1 = SelfTrainingClassifier(base_estimator, threshold=0.75, criterion='threshold')
semi_supervised_model1.fit(combined_features_case1, combined_labels_case1)

case1_predictions = semi_supervised_model1.predict_proba(testing_features)[:, 1]
case1_auc = roc_auc_score(testing_labels, case1_predictions)
print(f"Self-training (без тестовых): AUC-ROC = {case1_auc:.4f}")

del combined_features_case1, combined_labels_case1, semi_supervised_model1

print("\n3. Self-training с неаннотированными данными (тестовый набор как неразмеченный):")

combined_features_case2 = vstack([labeled_features, testing_features])
combined_labels_case2 = np.concatenate([labeled_labels, [-1] * testing_features.shape[0]])

semi_supervised_model2 = SelfTrainingClassifier(base_estimator, threshold=0.75, criterion='threshold')
semi_supervised_model2.fit(combined_features_case2, combined_labels_case2)

case2_predictions = semi_supervised_model2.predict_proba(testing_features)[:, 1]
case2_auc = roc_auc_score(testing_labels, case2_predictions)
print(f"Self-training (тест как неразмеченные): AUC-ROC = {case2_auc:.4f}")

del combined_features_case2, combined_labels_case2, semi_supervised_model2

print("\n4. Self-training с неаннотированными данными (тестовый набор включен):")

combined_features_case3 = vstack([labeled_features, unlabeled_features, testing_features])
combined_labels_case3 = np.concatenate([labeled_labels, [-1] * (unlabeled_features.shape[0] + testing_features.shape[0])])

semi_supervised_model3 = SelfTrainingClassifier(base_estimator, threshold=0.75, criterion='threshold')
semi_supervised_model3.fit(combined_features_case3, combined_labels_case3)

case3_predictions = semi_supervised_model3.predict_proba(testing_features)[:, 1]
case3_auc = roc_auc_score(testing_labels, case3_predictions)
print(f"Self-training (включая тест): AUC-ROC = {case3_auc:.4f}")

print("\nСВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
comparison_table = pd.DataFrame({
    'Экспериментальная схема': [
        'Только размеченные экземпляры',
        'Self-training (без тестовых)',
        'Self-training (тест как неразмеченные)',
        'Self-training (тестовые включены)'
    ],
    'Значение AUC-ROC': [
        baseline_auc,
        case1_auc,
        case2_auc,
        case3_auc
    ]
})
print(comparison_table.to_string(index=False))

print("\nАНАЛИТИЧЕСКОЕ ЗАКЛЮЧЕНИЕ:")

performance_change = "положительную динамику" if case1_auc > baseline_auc else "отрицательную динамику"
test_inclusion_effect = "демонстрирует искусственное преимущество" if case2_auc > baseline_auc else "не демонстрирует преимущества (некорректная методология!)"
optimal_condition = "неразмеченные данные изолированы от тестового набора" if case1_auc > baseline_auc else "улучшение достигается только при нарушении методологии"

print(f"""
1. Сопоставление с эталонной моделью:
   - Self-training с неаннотированными данными (без тестовых) достиг значения: {case1_auc:.4f}
   - Базовая модель на ограниченной разметке: {baseline_auc:.4f}
   - Дифференциал: {case1_auc - baseline_auc:.4f}

2. Воздействие тестовых данных:
   - При использовании тестовых как неаннотированных: {case2_auc:.4f}
   - При инкорпорировании тестовых в неаннотированный массив: {case3_auc:.4f}

3. Ключевые наблюдения:
   - Расширение выборки неаннотированными данными демонстрирует {performance_change}
   - Применение тестовых данных в роли неаннотированных {test_inclusion_effect}
   - Максимальная эффективность фиксируется при условии, что {optimal_condition}
""")

Файл train-sample.csv не найден!
Текущая директория: /content
Доступные CSV файлы: []

Создаю тестовые данные для демонстрации...

1. Обучение исключительно на аннотированных экземплярах:
Логистическая регрессия (только размеченные): AUC-ROC = 0.5000

2. Self-training с неаннотированными данными (исключая тестовый набор):
Self-training (без тестовых): AUC-ROC = 0.5000

3. Self-training с неаннотированными данными (тестовый набор как неразмеченный):
Self-training (тест как неразмеченные): AUC-ROC = 0.5000

4. Self-training с неаннотированными данными (тестовый набор включен):
Self-training (включая тест): AUC-ROC = 0.5000

СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:
               Экспериментальная схема  Значение AUC-ROC
         Только размеченные экземпляры               0.5
          Self-training (без тестовых)               0.5
Self-training (тест как неразмеченные)               0.5
     Self-training (тестовые включены)               0.5

АНАЛИТИЧЕСКОЕ ЗАКЛЮЧЕНИЕ:

1. Сопоставление с эталонной

Нас будут интересовать качество (AUC-ROC) в четырёх следующих постановках:
1. Модель обучается только на размеченных данных.
2. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть не пересекается с тестовой выборкой.
3. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть совпадает с тестовой выборкой.
4. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть включает в себя тестовую выборку.

**Задание 1. (5 баллов)**

Проведите эксперименты и сделайте выводы для любого из методов пакета `sklearn.semi_supervised` и для логистической регрессии

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier, LabelPropagation, LabelSpreading
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack
import warnings
import os
warnings.filterwarnings('ignore')

# Проверка наличия файла и загрузка данных
print("ПРОВЕРКА ДОСТУПНОСТИ ДАННЫХ")
file_path = 'train-sample.csv'

if os.path.exists(file_path):
    print(f"Файл {file_path} найден, загружаем...")
    data_frame = pd.read_csv(file_path)
else:
    print(f"Файл {file_path} не найден в директории: {os.getcwd()}")
    print("Доступные файлы:", [f for f in os.listdir() if f.endswith('.csv')])
    print("\nСОЗДАНИЕ СИНТЕТИЧЕСКИХ ДАННЫХ ДЛЯ ДЕМОНСТРАЦИИ")

    # Создаем синтетические данные
    np.random.seed(42)
    n_samples = 20000

    # Генерируем синтетические тексты
    texts = []
    words = ['python', 'code', 'error', 'help', 'function', 'variable', 'class', 'import',
             'debug', 'syntax', 'runtime', 'exception', 'array', 'list', 'dict', 'loop']

    for i in range(n_samples):
        text_length = np.random.randint(5, 20)
        text = ' '.join(np.random.choice(words, text_length))
        texts.append(text)

    # Генерируем статусы (open/closed)
    open_status = np.random.choice(['open', 'closed'], n_samples, p=[0.3, 0.7])

    data_frame = pd.DataFrame({
        'BodyMarkdown': texts,
        'OpenStatus': open_status
    })
    print(f"Создано {n_samples} синтетических записей")

print(f"Размер данных: {data_frame.shape}")
print("-" * 50)

# Создание целевой переменной
data_frame['target_class'] = (data_frame['OpenStatus'] == 'open').astype(int)
print(f"Распределение классов: \n{data_frame['target_class'].value_counts()}")

# Извлечение признаков
text_features = data_frame['BodyMarkdown'].fillna('')
target_values = data_frame['target_class']

# Векторизация текста
print("\nВекторизация текстовых данных...")
vectorizer = TfidfVectorizer(min_df=10, max_features=5000)
feature_matrix = vectorizer.fit_transform(text_features)
print(f"Размер матрицы признаков: {feature_matrix.shape}")

# Разделение на обучающую и тестовую выборки
train_features, test_features, train_labels, test_labels = train_test_split(
    feature_matrix, target_values, test_size=5000, random_state=42, stratify=target_values
)

# Разделение обучающей выборки на размеченную и неразмеченную части
labeled_count = 500
labeled_features, unlabeled_features, labeled_labels, unlabeled_labels = train_test_split(
    train_features, train_labels, train_size=labeled_count, random_state=42, stratify=train_labels
)

print(f"\nРазмер размеченной выборки: {labeled_features.shape[0]}")
print(f"Размер неразмеченной выборки: {unlabeled_features.shape[0]}")
print(f"Размер тестовой выборки: {test_features.shape[0]}")
print("-" * 50)

print("\nИССЛЕДОВАНИЕ ПОЛУНАДЗИРАЕМЫХ АЛГОРИТМОВ")

print("\n1. Базовый классификатор (только размеченная выборка):")
base_classifier = LogisticRegression(random_state=42, max_iter=1000)
base_classifier.fit(labeled_features, labeled_labels)
base_predictions = base_classifier.predict_proba(test_features)[:, 1]
base_auc = roc_auc_score(test_labels, base_predictions)
print(f"AUC-ROC = {base_auc:.6f}")

print("\n2. Алгоритм самообучения (Self-training):")
self_training_features = vstack([labeled_features, unlabeled_features])
self_training_labels = np.concatenate([labeled_labels, [-1] * unlabeled_features.shape[0]])

self_learning_model = SelfTrainingClassifier(
    LogisticRegression(random_state=42, max_iter=1000),
    threshold=0.75,
    criterion='threshold',
    max_iter=10
)
self_learning_model.fit(self_training_features, self_training_labels)
self_learning_predictions = self_learning_model.predict_proba(test_features)[:, 1]
self_learning_auc = roc_auc_score(test_labels, self_learning_predictions)
print(f"AUC-ROC = {self_learning_auc:.6f}")
print(f"Число выполненных итераций: {self_learning_model.n_iter_}")

print("\n3. Метод распространения меток (Label Propagation):")
# Для ускорения используем подвыборку
propagation_features = vstack([labeled_features[:300], unlabeled_features[:700]])
propagation_labels = np.concatenate([labeled_labels[:300], [-1] * 700])

label_propagation_model = LabelPropagation(kernel='knn', n_neighbors=7, max_iter=100)
label_propagation_model.fit(propagation_features, propagation_labels)
propagation_predictions = label_propagation_model.predict_proba(test_features)[:, 1]
propagation_auc = roc_auc_score(test_labels, propagation_predictions)
print(f"AUC-ROC = {propagation_auc:.6f}")

print("\n4. Алгоритм распространения с модификацией (Label Spreading):")
label_spreading_model = LabelSpreading(kernel='knn', n_neighbors=7, alpha=0.8, max_iter=100)
label_spreading_model.fit(propagation_features, propagation_labels)
spreading_predictions = label_spreading_model.predict_proba(test_features)[:, 1]
spreading_auc = roc_auc_score(test_labels, spreading_predictions)
print(f"AUC-ROC = {spreading_auc:.6f}")

print("\nСРАВНИТЕЛЬНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
comparison_results = pd.DataFrame({
    'Алгоритм': [
        'Логистическая регрессия (контрольный)',
        'Самообучение (Self-training)',
        'Распространение меток',
        'Модифицированное распространение'
    ],
    'Значение AUC-ROC': [base_auc, self_learning_auc, propagation_auc, spreading_auc]
})
print(comparison_results.to_string(index=False))

print("\nАНАЛИТИЧЕСКОЕ ЗАКЛЮЧЕНИЕ:")

optimal_approach = comparison_results.loc[comparison_results['Значение AUC-ROC'].idxmax(), 'Алгоритм']
optimal_score = comparison_results['Значение AUC-ROC'].max()
performance_gain = optimal_score - base_auc
gain_percentage = (performance_gain / base_auc * 100) if base_auc > 0 else 0

print(f"""
1. Сопоставление с эталонной моделью:
   - Контрольная логистическая регрессия: {base_auc:.6f}
   - Оптимальный полунадзираемый метод: {optimal_score:.6f} ({optimal_approach})
   - Прирост эффективности: {performance_gain:.6f} ({gain_percentage:.2f}%)

2. Детальный анализ алгоритмов:
   - Самообучение (Self-training): {self_learning_auc:.6f}
   - Распространение меток (Label Propagation): {propagation_auc:.6f}
   - Модифицированное распространение (Label Spreading): {spreading_auc:.6f}

3. Ключевое наблюдение:
   {'✓ Полунадзираемые подходы превосходят базовый классификатор' if optimal_score > base_auc else '✗ Полунадзираемые методы не продемонстрировали преимущества'}
""")

ПРОВЕРКА ДОСТУПНОСТИ ДАННЫХ
Файл train-sample.csv не найден в директории: /content
Доступные файлы: []

СОЗДАНИЕ СИНТЕТИЧЕСКИХ ДАННЫХ ДЛЯ ДЕМОНСТРАЦИИ
Создано 20000 синтетических записей
Размер данных: (20000, 2)
--------------------------------------------------
Распределение классов: 
target_class
0    13967
1     6033
Name: count, dtype: int64

Векторизация текстовых данных...
Размер матрицы признаков: (20000, 16)

Размер размеченной выборки: 500
Размер неразмеченной выборки: 14500
Размер тестовой выборки: 5000
--------------------------------------------------

ИССЛЕДОВАНИЕ ПОЛУНАДЗИРАЕМЫХ АЛГОРИТМОВ

1. Базовый классификатор (только размеченная выборка):
AUC-ROC = 0.496055

2. Алгоритм самообучения (Self-training):
AUC-ROC = 0.498032
Число выполненных итераций: 9

3. Метод распространения меток (Label Propagation):
AUC-ROC = 0.510121

4. Алгоритм распространения с модификацией (Label Spreading):
AUC-ROC = 0.512179

СРАВНИТЕЛЬНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:
                             Ал

### Self-train

Обучаем на размеченной части, предсказываем неразмеченную, потом обучаем на всех, и предсказываем неразмеченную, повторяем пока не сойдёмся в предскзааниях неразмеченной части

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack
import warnings
import os
warnings.filterwarnings('ignore')

print("ПОДГОТОВКА ДАННЫХ ДЛЯ ЭКСПЕРИМЕНТА")

# Проверка наличия файла
file_path = 'train-sample.csv'

if os.path.exists(file_path):
    print(f"Загрузка данных из {file_path}")
    data_container = pd.read_csv(file_path)
else:
    print(f"Файл {file_path} не найден. Создание синтетических данных...")
    np.random.seed(42)
    n_samples = 15000
    text_templates = [
        "как исправить ошибку в коде", "проблема с установкой библиотеки",
        "не работает функция", "ошибка компиляции", "помощь с алгоритмом",
        "баг в программе", "непонятное исключение", "сложности с синтаксисом",
        "как оптимизировать код", "вопрос по архитектуре"
    ]
    synthetic_texts = np.random.choice(text_templates, n_samples)
    synthetic_status = np.random.choice(['open', 'closed'], n_samples, p=[0.35, 0.65])
    data_container = pd.DataFrame({
        'BodyMarkdown': synthetic_texts,
        'OpenStatus': synthetic_status
    })

# Подготовка данных
print("Векторизация текстовых данных...")
data_container['binary_outcome'] = (data_container['OpenStatus'] == 'open').astype(int)

text_sequences = data_container['BodyMarkdown'].fillna('')
target_sequence = data_container['binary_outcome']

vectorization_engine = TfidfVectorizer(min_df=10, max_features=5000)
feature_space = vectorization_engine.fit_transform(text_sequences)

# Разделение на обучающую и тестовую выборки
print("Разделение данных...")
train_attributes, test_attributes, train_labels, test_labels = train_test_split(
    feature_space, target_sequence, test_size=5000, random_state=42, stratify=target_sequence
)

# Разделение на размеченную и неразмеченную части
labeled_pool_size = 500
labeled_attributes, unlabeled_attributes, labeled_labels, unlabeled_labels = train_test_split(
    train_attributes, train_labels, train_size=labeled_pool_size, random_state=42, stratify=train_labels
)

print(f"Размеченных экземпляров: {labeled_attributes.shape[0]}")
print(f"Неразмеченных экземпляров: {unlabeled_attributes.shape[0]}")
print(f"Тестовых экземпляров: {test_attributes.shape[0]}")
print("="*60)

print("ИМПЛЕМЕНТАЦИЯ АЛГОРИТМА САМООБУЧЕНИЯ")

def iterative_self_learning(labeled_features, labeled_targets, unlabeled_features, test_features, test_targets,
                           confidence_threshold=0.75, max_rounds=10):
    """
    Ручная реализация алгоритма самообучения для полунадзираемого обучения
    """
    current_train_features = labeled_features.copy()
    current_train_targets = labeled_targets.copy()
    remaining_unlabeled = unlabeled_features.copy()

    previous_test_predictions = None

    print(f"Начальная конфигурация:")
    print(f"  Аннотированных экземпляров: {current_train_features.shape[0]}")
    print(f"  Неразмеченных экземпляров: {remaining_unlabeled.shape[0]}")
    print(f"  Порог достоверности: {confidence_threshold}")
    print()

    for iteration in range(max_rounds):
        # Обучение модели на текущем наборе данных
        base_estimator = LogisticRegression(random_state=42, max_iter=1000)
        base_estimator.fit(current_train_features, current_train_targets)

        # Оценка на тестовых данных
        test_probabilities = base_estimator.predict_proba(test_features)[:, 1]

        # Проверка сходимости
        if previous_test_predictions is not None:
            prediction_drift = np.mean(np.abs(test_probabilities - previous_test_predictions))
            print(f"  Дрейф предсказаний: {prediction_drift:.6f}")
            if prediction_drift < 0.001:
                print(f"  Сходимость достигнута на раунде {iteration + 1}")
                break

        previous_test_predictions = test_probabilities

        # Добавление новых размеченных экземпляров
        if remaining_unlabeled.shape[0] > 0:
            # Предсказание для неразмеченных данных
            unlabeled_probabilities = base_estimator.predict_proba(remaining_unlabeled)
            unlabeled_predictions = base_estimator.predict(remaining_unlabeled)
            prediction_confidence = np.max(unlabeled_probabilities, axis=1)

            # Отбор уверенных предсказаний
            confident_mask = prediction_confidence >= confidence_threshold

            if np.sum(confident_mask) == 0:
                print(f"  Нет уверенных предсказаний на раунде {iteration + 1}")
                break

            # Выделение уверенных экземпляров
            confident_features = remaining_unlabeled[confident_mask]
            confident_targets = unlabeled_predictions[confident_mask]

            # Обновление обучающих наборов
            current_train_features = vstack([current_train_features, confident_features])
            current_train_targets = np.concatenate([current_train_targets, confident_targets])
            remaining_unlabeled = remaining_unlabeled[~confident_mask]

            print(f"Раунд {iteration + 1}:")
            print(f"  Добавлено экземпляров: {np.sum(confident_mask)}")
            print(f"  Текущий размер обучающей выборки: {current_train_features.shape[0]}")
            print(f"  Осталось неразмеченных: {remaining_unlabeled.shape[0]}")
            print(f"  Средняя достоверность добавленных: {prediction_confidence[confident_mask].mean():.4f}")
        else:
            print(f"  Все экземпляры размечены на раунде {iteration + 1}")
            break
        print()

    print("Финальное обучение модели...")
    final_classifier = LogisticRegression(random_state=42, max_iter=1000)
    final_classifier.fit(current_train_features, current_train_targets)

    # Финальная оценка
    final_predictions = final_classifier.predict_proba(test_features)[:, 1]
    final_auc = roc_auc_score(test_targets, final_predictions)

    print(f"\nИтоговые метрики самообучения:")
    print(f"  Конечный размер обучающей выборки: {current_train_features.shape[0]}")
    print(f"  Количество выполненных раундов: {iteration + 1}")
    print(f"  AUC-ROC на тестовом наборе: {final_auc:.6f}")

    return final_auc, iteration + 1, current_train_features.shape[0]

print("\n" + "="*60)
print("СЦЕНАРИЙ 1: Порог достоверности = 0.75")
auc_case1, rounds_case1, samples_case1 = iterative_self_learning(
    labeled_attributes, labeled_labels, unlabeled_attributes, test_attributes, test_labels,
    confidence_threshold=0.75, max_rounds=10
)

print("\n" + "="*60)
print("СЦЕНАРИЙ 2: Порог достоверности = 0.9")
auc_case2, rounds_case2, samples_case2 = iterative_self_learning(
    labeled_attributes, labeled_labels, unlabeled_attributes, test_attributes, test_labels,
    confidence_threshold=0.9, max_rounds=10
)

print("\n" + "="*60)
print("СЦЕНАРИЙ 3: Порог достоверности = 0.5")
auc_case3, rounds_case3, samples_case3 = iterative_self_learning(
    labeled_attributes, labeled_labels, unlabeled_attributes, test_attributes, test_labels,
    confidence_threshold=0.5, max_rounds=10
)

print("\n" + "="*60)
print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ")

initial_sample_size = labeled_attributes.shape[0]

comparison_dataframe = pd.DataFrame({
    'Порог': [0.5, 0.75, 0.9],
    'Раунды': [rounds_case3, rounds_case1, rounds_case2],
    'Финальный размер': [samples_case3, samples_case1, samples_case2],
    'Добавлено': [samples_case3 - initial_sample_size,
                  samples_case1 - initial_sample_size,
                  samples_case2 - initial_sample_size],
    'AUC-ROC': [auc_case3, auc_case1, auc_case2]
})
print(comparison_dataframe.to_string(index=False))

print("\nАНАЛИТИЧЕСКИЕ ВЫВОДЫ ПО АЛГОРИТМУ САМООБУЧЕНИЯ:")

optimal_threshold_idx = comparison_dataframe['AUC-ROC'].idxmax()
optimal_threshold = comparison_dataframe.loc[optimal_threshold_idx, 'Порог']
max_auc = comparison_dataframe['AUC-ROC'].max()

print(f"""
1. Влияние порога достоверности на эффективность:
   - Консервативный порог (0.5): AUC = {auc_case3:.6f}, пополнение = {samples_case3 - initial_sample_size} экз.
   - Умеренный порог (0.75): AUC = {auc_case1:.6f}, пополнение = {samples_case1 - initial_sample_size} экз.
   - Строгий порог (0.9): AUC = {auc_case2:.6f}, пополнение = {samples_case2 - initial_sample_size} экз.

2. Оптимальная конфигурация:
   - Наилучший порог достоверности: {optimal_threshold}
   - Максимальное значение AUC-ROC: {max_auc:.6f}

3. Закономерности процесса обучения:
   - Низкий порог: интенсивное пополнение выборки, потенциальное зашумление
   - Высокий порог: селективное добавление, замедленное обучение
   - Сбалансированный порог: компромисс между объемом и качеством

4. Итоговое заключение:
   {'✓ Алгоритм самообучения демонстрирует эффективность' if max_auc > 0.74 else '✗ Алгоритм самообучения не показал значимого улучшения'}
   Наилучший достигнутый результат: {max_auc:.6f}
""")

ПОДГОТОВКА ДАННЫХ ДЛЯ ЭКСПЕРИМЕНТА
Файл train-sample.csv не найден. Создание синтетических данных...
Векторизация текстовых данных...
Разделение данных...
Размеченных экземпляров: 500
Неразмеченных экземпляров: 9500
Тестовых экземпляров: 5000
ИМПЛЕМЕНТАЦИЯ АЛГОРИТМА САМООБУЧЕНИЯ

СЦЕНАРИЙ 1: Порог достоверности = 0.75
Начальная конфигурация:
  Аннотированных экземпляров: 500
  Неразмеченных экземпляров: 9500
  Порог достоверности: 0.75

Раунд 1:
  Добавлено экземпляров: 930
  Текущий размер обучающей выборки: 1430
  Осталось неразмеченных: 8570
  Средняя достоверность добавленных: 0.7739

  Дрейф предсказаний: 0.025863
  Нет уверенных предсказаний на раунде 2
Финальное обучение модели...

Итоговые метрики самообучения:
  Конечный размер обучающей выборки: 1430
  Количество выполненных раундов: 2
  AUC-ROC на тестовом наборе: 0.489147

СЦЕНАРИЙ 2: Порог достоверности = 0.9
Начальная конфигурация:
  Аннотированных экземпляров: 500
  Неразмеченных экземпляров: 9500
  Порог достоверности: 